# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [1]:
%%bash

source ~/.bash_profile

#### Check environment variables are set with valid values
echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

#### Download the `kfctl` CLI tool
curl --location https://github.com/kubeflow/kfctl/releases/download/v1.0.2/kfctl_v1.0.2-0-ga476281_linux.tar.gz | tar xz
sudo mv kfctl /usr/local/bin

#### Get the latest Kubeflow configuration file

# With Ingress
export CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml"
#export CONFIG_URI='https://raw.githubusercontent.com/kubeflow/manifests/v1.0.2/kfdef/kfctl_aws.v1.0.2.yaml'
echo "export CONFIG_URI=${CONFIG_URI}" | tee -a ~/.bash_profile

#### Set Kubeflow environment variables 
export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

#cd ~/SageMaker/kubeflow/notebooks/part-3-kubernetes

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

#### Customize the configuration files
# We'll edit the configuration with the right names for the cluster and node groups before deploying Kubeflow.

mkdir -p ${KF_DIR}
cd ${KF_DIR}

curl -O ${CONFIG_URI}

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

#### Generate the Kubeflow installation files
cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

#### Deploy Kubeflow
cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

#### Remove istio-ingress in --namespace kubeflow
kubectl delete ingress istio-ingress --namespace kubeflow

#### Deploy istio-ingress in --namespace istio-system
cd ${KF_DIR}/.cache/manifests/manifests-1.0.2/ 
kubectl apply -k aws/istio-ingress/base --namespace istio-system
cd ${KF_DIR}

#### Delete the usage reporting beacon
kubectl delete all -l app=spartakus --namespace=kubeflow

#### Change to `kubeflow` namespace
kubectl config set-context --current --namespace=kubeflow

sagemaker-us-west-2-853609162619
chubacluster


export CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml
export KF_NAME=chubacluster
export KF_DIR=/home/ec2-user/SageMaker/workshop/11_kubeflow/chubacluster
export CONFIG_FILE=/home/ec2-user/SageMaker/workshop/11_kubeflow/chubacluster/kfctl_aws.v1.0.2.yaml
customresourcedefinition.apiextensions.k8s.io/adapters.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/apikeys.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/attributemanifests.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/authorizations.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/bypasses.config.istio.io created
customresourcedefinition.apiextensions.k8s.io/certificates.certmanager.k8s.io created
customresourcedefinition.apiextensions.k8s.io/challenges.certmanager.k8s.io created
customresourcedefinition.apiextensions.k8s.io/checknothings.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   649    0     0  12725      0 --:--:-- --:--:-- --:--:-- 12725
100 27.9M  100 27.9M    0     0  12.1M      0  0:00:02  0:00:02 --:--:-- 12.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8801  100  8801    0     0   190k      0 --:--:-- --:--:-- --:--:--  190k
time="2020-05-23T22:12:44Z" level=info msg="No name specified in KfDef.Metadata.Name; defaulting to chubacluster based on location of config file: /home/ec2-user/SageMaker/workshop/11_kubeflow/chubacluster/kfctl_aws.v1.0.2.yaml." filename="coordinator/coordinator.go:202"
time="2020-05-23T22:12:44Z" level=info msg="Running `eksctl version` ..." filename="utils/awsutil.go:71"
time="2020-05-23T22:12:44Z" level=info msg="eksctl version: 0.19.0" fi

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [2]:
!kubectl -n kubeflow get all


NAME                                                               READY   STATUS              RESTARTS   AGE
pod/admission-webhook-bootstrap-stateful-set-0                     1/1     Running             0          11s
pod/admission-webhook-deployment-569558c8b6-6cqht                  0/1     Terminating         0          11s
pod/admission-webhook-deployment-569558c8b6-zjznz                  0/1     ContainerCreating   0          2s
pod/alb-ingress-controller-976d67f4f-8znpg                         0/1     ContainerCreating   0          2s
pod/application-controller-stateful-set-0                          1/1     Running             0          40s
pod/argo-ui-7ffb9b6577-wcsm2                                       0/1     ContainerCreating   0          11s
pod/centraldashboard-659bd78c-trdbz                                0/1     ContainerCreating   0          11s
pod/jupyter-web-app-deployment-679d5f5dc4-7lmcz                    0/1     ContainerCreating   0          11s
pod/katib-co

In [3]:
!kubectl -n istio-system get all


NAME                                          READY   STATUS              RESTARTS   AGE
pod/cluster-local-gateway-7bf56777fb-ptvth    0/1     Running             0          42s
pod/grafana-86f89dbd84-bjt4t                  1/1     Running             0          44s
pod/istio-citadel-74966f47d6-2b9gv            1/1     Running             0          44s
pod/istio-cleanup-secrets-1.1.6-gx6g2         0/1     Completed           0          43s
pod/istio-egressgateway-5c64d575bc-859x2      0/1     Running             0          44s
pod/istio-galley-784b9f6d75-p72w8             0/1     ContainerCreating   0          44s
pod/istio-grafana-post-install-1.1.6-b8qwb    0/1     Completed           0          43s
pod/istio-ingressgateway-589ff776dd-xt5v4     0/1     Running             0          44s
pod/istio-pilot-677df6b6d4-fpck9              1/2     Running             0          44s
pod/istio-policy-6f74d9d95d-tgbmc             2/2     Running             1          43s
pod/istio-security-po